In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)

from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner

In [ ]:
gs.destroy()
gs.init(backend=gs.cpu)
gs.set_random_seed(seed=42)
#config = {'viewr_option' : {'width': 640, 'height': 480}, 'box_box_detection': True}
adapter = GenesisAdapter()
scene = adapter.scene

In [ ]:
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='../model/panda_vir.xml'),
)
                          
# table = scene.add_entity(
#     gs.morphs.MJCF(file='../model/common_arena/tables.xml'),
# )

table1 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0 ,
        friction=5.0
    )
)
        
table2 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (-0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table3 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, -0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table4 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, 0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

bottle = scene.add_entity(
    material=gs.materials.Rigid(rho=300),
    morph=gs.morphs.URDF(
        file="urdf/3763/mobility_vhacd.urdf",
        scale=0.09,
        pos=(0.65, 0.0, 0.036 + 0.19),
        quat=(-np.pi/2, 0, np.pi/2, 0),
    ),
    # visualize_contact=True,
)

# bottle = scene.add_entity(
#     morph = gs.morphs.URDF(
#         file="../assets/urdf/bottle/bottle.urdf",
#         scale=0.1,
#         pos=(0.65, 0.4, 0.036 + 0.19),
#         quat=(-np.pi/2, 0, np.pi/2, 0),
#     ),
# )
cube_size = (0.07, 0.1, 0.1)
cube_pos = (0.4, 0.4, 0.05+0.19)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
    ),
    surface=gs.surfaces.Default(
        color = (0.8, 0.1 , 0.1),   
    )
)
cube.surface.color = (0.8, 0.1 , 0.1)

cylinder_pos = (0.7, -0.2, 0.03+0.19)
cylinder_quat = (0, -np.sqrt(2)/2, 0, np.sqrt(2)/2)
cylinder_radius = 0.03
cylinder_height = 0.15

cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius = cylinder_radius,
        height = cylinder_height,
        pos    = cylinder_pos,
        quat   = cylinder_quat,
    ),
    surface=gs.surfaces.Default(
        color = (0.1, 0.8 , 0.1),   
    )
)
cylinder.surface.color = (0.2, 0.8, 0.2)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)

In [ ]:
n_envs = 9
scene.build(n_envs=n_envs, env_spacing=(2, 2), center_envs_at_origin=True )

In [ ]:
import random

def random_dual_range(range1,range2):
    a, b = range1
    c, d = range2
    if random.random() < 0.5:
        return np.random.uniform(a, b)
    else:
        return np.random.uniform(c, d)

In [ ]:
cube.get_pos()

In [ ]:
cube.get_pos(1)

In [ ]:
cube.set_pos((1, 2, 3), envs_idx=0)
scene.step()

In [ ]:
scene.reset()

In [ ]:
adapter.set_pose(cube, np.array([0,0,0,1,0,0,0]), envs_idx=0)

In [ ]:
scene.reset()
for env in range(n_envs):
    cube.set_pos((random_dual_range([0.25,0.45],[-0.25,-0.45]), random_dual_range([0.35,0.55],[-0.35,-0.55]), 0.05+0.19), envs_idx=env)
    cylinder.set_pos((random_dual_range([0.25,0.45],[-0.25,-0.45]), random_dual_range([0.35,0.55],[-0.35,-0.55]), 0.03+0.19), envs_idx=env)
    scene.step()

In [ ]:
adapter.set_pose(cube,[random_dual_range([0.25,0.45],[-0.25,-0.45]), random_dual_range([0.35,0.55],[-0.35,-0.55]), 0.05+0.19], envs_idx=0)